In [1]:
import pyodbc
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import hashlib
from cryptography.fernet import Fernet
import pandas as pd
import seaborn as sns
import pickle
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
plt.style.use('fivethirtyeight')

In [ ]:
# sql Connection String
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=22.247.13.89;DATABASE=SSISRND;UID=test;PWD=test')
cursor = cnxn.cursor()

production_Qty_sql ="""SELECT [Production Id],[Production Date],[Unit Id],[WorkCenter Id],[WorkCenter Name]
                    ,[Item Id],[Item Name],[UOM Name],sum([Quantity]) as [Production_Quantity] FROM [SSISRND].[mes].[tblProductionInfo]
                    where [Unit Id]=4 and [UOM Id]=132 and [is Approved]=1
                    group by [Production Date],[WorkCenter Id],[WorkCenter Name],[Item Id],[Item Name],[Unit Id],[UOM Name],[Production Id]
                    order by [Production Date]"""

In [3]:
delivery_Qty_sql =""" SELECT cast([Delivery_Date] as date)  as [Production Date] ,Item_ID as [Item Id],[Item_Name] as [Item Name],sum([Delivery_Qty]) as [Delivery Qty]
FROM [SSISRND].[o2c].[DeliveryInformation] as d
  join ARLEDW.dbo.ItemProfile as i on i.[Item id] = d.Item_ID
  where is_Shipment_Complete = 1 and unit_id=4 and [Base Uom Name]='Bag'
  group by [Delivery_Date],Item_ID,[Item_Name]
  order by Delivery_Date"""  


In [4]:
production_Qty_sql =""" SELECT [Production Date],[Item Id],[Item Name],sum([Quantity]) as [Production_Quantity] FROM [SSISRND].[mes].[tblProductionInfo]
                    where [Unit Id]=4 and [UOM Id]=132 and [is Approved]=1
                    group by [Production Date],[Item Id],[Item Name]
                    order by [Production Date]"""  
    


In [5]:
# Load Production data
production_Qty_data = pd.read_sql(production_Qty_sql,cnxn)

In [6]:
# Load Delivery info data
delivery_Qty_data = pd.read_sql(delivery_Qty_sql,cnxn)

In [7]:
# remove 0 production value 
production_Qty_data = production_Qty_data[production_Qty_data['Production_Quantity'] != 0]


In [8]:
# Impute missing values with either previous or next non-null values based on the date
production_Qty_data['Production_Quantity'] = production_Qty_data['Production_Quantity'].fillna(method='ffill').fillna(method='bfill')


In [9]:
# Assuming production_Qty_data is your DataFrame
production_Qty_data['Production Date'] = pd.to_datetime(production_Qty_data['Production Date'])

delivery_Qty_data['Production Date']=pd.to_datetime(delivery_Qty_data['Production Date'])

In [10]:
merged_data = pd.merge(delivery_Qty_data,production_Qty_data, on=['Item Id', 'Production Date'], how='inner')
production_Qty_data=merged_data

In [11]:
# Remove 'Item Name_y' column
if 'Item Name_y' in production_Qty_data.columns:
    # Remove 'Item Name_y' column
    production_Qty_data.drop('Item Name_y', axis=1, inplace=True)


# Rename 'Item Name_x' to 'Item Name'
if 'Item Name_x' in production_Qty_data.columns:
    # Rename 'Item Name_x' to 'Item Name'
    production_Qty_data.rename(columns={'Item Name_x': 'Item Name'}, inplace=True)


In [12]:
production_Qty_data

,Production Date,Item Id,Item Name,Delivery Qty,Production_Quantity
0,2022-01-22,38935,Akij Cement (Bag)-PCC-1ply,69942.0,75345.0
1,2022-01-23,38935,Akij Cement (Bag)-PCC-1ply,123578.0,50075.0
2,2022-01-23,38931,Akij Cement (Bag)-OPC,4842.0,3640.0
3,2022-01-24,38931,Akij Cement (Bag)-OPC,6250.0,4450.0
4,2022-01-24,38935,Akij Cement (Bag)-PCC-1ply,144620.0,43975.0
...,...,...,...,...,...
2339,2024-02-09,38933,Akij Cement (Bag)-CEM-II (A-M),750.0,750.0
2340,2024-02-09,38935,Akij Cement (Bag)-PCC-1ply,35146.0,33379.0
2341,2024-02-10,38935,Akij Cement (Bag)-PCC-1ply,42823.0,46840.0
2342,2024-02-10,102630,Akij Cement (Bag)-PCC-2ply,52914.0,49974.0


In [13]:
# Extract features from 'Production Date'
production_Qty_data['Month'] = production_Qty_data['Production Date'].dt.month
production_Qty_data['Day'] = production_Qty_data['Production Date'].dt.day
production_Qty_data['DayOfWeek'] = production_Qty_data['Production Date'].dt.dayofweek

In [14]:
import pandas as pd
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Assuming production_Qty_data is your DataFrame with historical production data
# Replace these with your actual DataFrame

# Add 'Delivery Qty' to the features
features = ['Month', 'Day', 'DayOfWeek', 'Delivery Qty']

# Create a DataFrame with future dates for the next 30 days
future_dates = pd.date_range(start=production_Qty_data['Production Date'].max() + timedelta(days=1), periods=30, freq='D')
future_data = pd.DataFrame({'Production Date': future_dates})

# Initialize a dictionary to store future predictions for each item and model
future_predictions_dict = {model_name: {} for model_name in ['Linear Regression', 'Decision Tree', 'Gradient Boosting', 'Random Forest']}

# Iterate over unique items and models to make predictions for the future dates
for model_name, model in [('Linear Regression', LinearRegression()),
                          ('Decision Tree', DecisionTreeRegressor()),
                          ('Gradient Boosting', GradientBoostingRegressor()),
                          ('Random Forest', RandomForestRegressor())]:

    for item_id, item_data in production_Qty_data.groupby('Item Id'):
        # Filter future data for the current item
        future_item_data = future_data.copy()
        future_item_data['Month'] = future_item_data['Production Date'].dt.month
        future_item_data['Day'] = future_item_data['Production Date'].dt.day
        future_item_data['DayOfWeek'] = future_item_data['Production Date'].dt.dayofweek

        # Add 'Delivery Qty' as a feature
        future_item_data['Delivery Qty'] = item_data['Delivery Qty'].mean()

        # Prepare training data for the current item
        X_train = item_data[features]
        y_train = item_data['Production_Quantity']

        # Fit the model for the current item
        model.fit(X_train, y_train)

        # Make predictions for the future dates using the trained model
        future_predictions = model.predict(future_item_data[features])

        # Store the predictions in the dictionary for the current item and model
        future_predictions_dict[model_name][item_id] = future_predictions

# Create a DataFrame to store the consolidated results
consolidated_result_df = pd.DataFrame()

# Iterate over models
for model_name, model_predictions_dict in future_predictions_dict.items():
    # Initialize a DataFrame to store the results for the current model
    result_df = pd.DataFrame()

    # Iterate over items and concatenate predictions to the result DataFrame
    for item_id, predictions in model_predictions_dict.items():
        item_predictions_df = pd.DataFrame({
            'Item ID': [item_id] * len(future_dates),
            'Production Date': future_dates,
            'Predicted_Production_Quantity': predictions
        })
        result_df = pd.concat([result_df, item_predictions_df], ignore_index=True)

    # Add an additional column for the model name
    result_df['Model'] = model_name

    # Display the forecasted production quantity for the next 30 days for the current model
    print(f"\nResults for {model_name}:")
    print(result_df)

    # Export the DataFrame to CSV
    output_csv_path = f'result_{model_name.replace(" ", "_")}.csv'
    result_df.to_csv(output_csv_path, index=False)
    print(f"Output result CSV for {model_name}: {output_csv_path}")

    # Concatenate the results to the consolidated DataFrame
    consolidated_result_df = pd.concat([consolidated_result_df, result_df], ignore_index=True)

# Display the consolidated result DataFrame
print("\nConsolidated Result DataFrame:")
print(consolidated_result_df)

# Export the consolidated DataFrame to a CSV file
consolidated_output_csv_path = 'consolidated_result.csv'
consolidated_result_df.to_csv(consolidated_output_csv_path, index=False)
print(f"Output consolidated result CSV: {consolidated_output_csv_path}")



Results for Linear Regression:
     Item ID Production Date  Predicted_Production_Quantity              Model
0      38931      2024-02-11                    4670.445952  Linear Regression
1      38931      2024-02-12                    4729.198096  Linear Regression
2      38931      2024-02-13                    4689.321123  Linear Regression
3      38931      2024-02-14                    4649.444150  Linear Regression
4      38931      2024-02-15                    4609.567177  Linear Regression
..       ...             ...                            ...                ...
115   102630      2024-03-07                   54581.789454  Linear Regression
116   102630      2024-03-08                   55340.361199  Linear Regression
117   102630      2024-03-09                   56098.932944  Linear Regression
118   102630      2024-03-10                   56857.504689  Linear Regression
119   102630      2024-03-11                   52090.946273  Linear Regression

[120 rows x 4 colum

In [15]:

# table_name = 'pre.tblPredictedProduction'
# current_datetime = datetime.now()
# intUnitID=144
# cursor.execute(f"TRUNCATE TABLE {table_name}")
# # Insert the DataFrame into the SQL Server database
# try:
#     for index, row in consolidated_result_df.iterrows():
#         values = (row['Item ID'], row['Production Date'], row['Predicted_Production_Quantity'], row['Model'])
#         cursor.execute(f"INSERT INTO {table_name} (Item_ID, Production_Date, Predicted_Production_Quantity, Model) VALUES (?, ?, ?, ?)", values)
    
#     cnxn.commit()
#     print(f"Data has been successfully inserted into the SQL Server table: {table_name}")

# except Exception as e:
#     print(f"Error: {e}")

# finally:
#     # Close the cursor and connection
#     # cursor.close()
#     # cnxn.close()
#     print("connection Close")

In [16]:
from datetime import datetime
import time

# Assuming consolidated_result_df is the DataFrame you want to insert
# Replace these with your actual DataFrame and database connection information

table_name = 'pre.tblPredictedProduction'
current_datetime = datetime.now()
intUnitID = 4

try:
    # Truncate the table before the MERGE operation
    cursor.execute(f"TRUNCATE TABLE {table_name}")

    # Iterate through each row in the DataFrame and perform insert or update (replace) using MERGE statement
    for index, row in consolidated_result_df.iterrows():
        cursor.execute(f"""
            INSERT INTO {table_name} (intUnitID, Item_ID, Production_Date, Predicted_Production_Quantity, Model, dteInsertDate)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (intUnitID, row['Item ID'], row['Production Date'], row['Predicted_Production_Quantity'], row['Model'], current_datetime))

    cnxn.commit()  # Commit after the insert operation


    print("Data insertion or replacement completed.")

    print("Sleeping for 2 hours...")
    time.sleep(2 * 60 * 60)  # 2 hours in seconds

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    cnxn.close()


Data insertion or replacement completed.
Sleeping for 2 hours...
